![HPEDEVlogo](Pictures/hpedevlogo-NB.JPG)    ![Dockerlogo](Pictures/docker-compose.png)  

In [ ]:
%login {{ hostvars[inventory_hostname]['IP-WKSHP-Docker101'] }}

# Using Docker compose

Docker compose is a tool part of the Docker ecosystem.
It is used to run solutions split in multiple containers which is the case most of the time.
This is mainly due to the Docker philosophy to use one container per service.

Another benefit is to define the container running parameters within a YAML configuration file.

## Our first docker-compose.yml file
We need to create an application environment and our first **docker-compose.yml** configuration file.

Use the build environment of the previous chapter and create our configuration file in it. 

Note : There are various docker-compose binaries and configuration file format existing.You can find support information [here](https://docs.docker.com/compose/release-notes/)

In [ ]:
export STUDENTHOME=`pwd`
export HTTPPORT=`cat HTTPPORT`
cd build
cat > docker-compose.yml << EOF
version: '3.5'
services:
  hpedevcloud_web:
    build: .
    image: img_hpedevcloud_web
    volumes:
      - $STUDENTHOME/data:/var/www/html/nextcloud/data
      - $STUDENTHOME/config:/var/www/html/nextcloud/config
    ports:
      - "$HTTPPORT:80"
EOF

The above file asks to docker-compose to define a web service that will be built from our Dockerfile, to map port $HTTPPORT to 80 and to map the data and config directories at their expected place in the container.

We can now start our application using:

In [ ]:
docker-compose up --build -d

In [ ]:
docker container ls

Our application starts and should work the same way as previously. However it is much simpler because we don't need to define ports and storage mapping using the command line, also the YAML file can be held in and this information can be managed in a Configuration Management System such as `git`.

You can also note that the container name is defined as `context_service_number` (build_hpedevcloud_web_1)

Test your nexcloud instance as previously in your browser.

Check what happens to the container.

In [4]:
docker container ls -a

Now stop the application and look at the containers again

In [ ]:
docker-compose down
docker container ls -a

So contrary to the basic `docker` command, `docker-compose` does the cleanup of containers after shutdown. 

Ok that's cool, but it is not really a big change so far.

## Going further with docker-compose.yml

If we look at our nextcloud application, we see we are using an internal sqlite database. This was defined during the setup phase.

As mentioned during the setup (below), this is convenient for a limited installation, but for larger ones it is better to use mariadb or postgres.

![Nextcloud sqlite setup](Pictures/nextcloud_setup.png)

In order to install nextcloud with another database: 

   1. Cleanup the nexcloud install in order to have the setup page proposed again by the application.
   2. Add the `php-mysqlnd` package to your Dockerfile when installing dependencies for nexcloud.
   3. Add a new container running a MariaDB server to store our data in a persistent manner, using a private network to communicate between the frontend and the backend.
   4. Start the application stack forcing the rebuild of the image as the Dockerfile changed.
   5. Look at the result

First, instead of building our own mariadb container from scratch like we did for nextcloud, we will use the official Docker one.

Of course it requires some information about the compose-file format. Documentation for this can be found here: https://docs.docker.com/compose/compose-file and the image itself there: https://hub.docker.com/_/mariadb
We will pass some variables to the container creation in order to be able to log on it afterwards.

In [ ]:
docker-compose up -d
docker exec build_hpedevcloud_web_1 rm -rf /var/www/html/nextcloud/data
docker exec build_hpedevcloud_web_1 rm -f /var/www/html/nextcloud/config/config.php
docker exec build_hpedevcloud_web_1 touch /var/www/html/nextcloud/config/CAN_INSTALL
mkdir -p ../db
docker-compose down
sed -i -e 's/php-zip/php-zip php-mysqlnd/' Dockerfile
export STUDENTHOME=`dirname $(pwd)`

cat >> docker-compose.yml << EOF
    networks:
      - nclan
    depends_on:
      - hpedevcloud_db
      
  hpedevcloud_db:
    image: mariadb
    # Hint from https://techoverflow.net/2021/08/17/how-to-fix-nextcloud-4047-innodb-refuses-to-write-tables-with-row_formatcompressed-or-key_block_size/
    command: --transaction-isolation=READ-COMMITTED --binlog-format=ROW --innodb-file-per-table=1 --skip-innodb-read-only-compressed
    restart: always
    environment:
      - MYSQL_ROOT_PASSWORD=password
      - MYSQL_DATABASE=nextcloud
      - MYSQL_USER=nextcloud
      - MYSQL_PASSWORD=nextcloudpwd
    networks:
      - nclan
    volumes:
      - $STUDENTHOME/db:/var/lib/mysql
      
networks:
  nclan:
    driver: bridge
EOF
docker-compose up --build -d
docker-compose ps

![Nextcloud MariaDB setup](Pictures/nextcloud_db_setup.png)

The password to use is the one passed with the `MYSQL_PASSWORD` environment variable.

Check that you can have access to this newly configured Nextcloud based on MariaDB.
First if you have a gateway issue/timeout, refresh the page (Shift reload may be necessary). 

Log in back and start using again by uploading a file. Stop and restart again the stack and check that the file is still there.

In [ ]:
docker-compose down
docker-compose up -d
docker-compose ps

Great ! 

As a next step, we could manage our own MariaDB server from the same standard Linux distribution to allow for common base. In that case, you'd probably have to launch the server using: `CMD /usr/libexec/mysql-check-socket && /usr/libexec/mysql-prepare-db-dir && /usr/libexec/mysql-check-socket && /usr/libexec/mysql-prepare-db-dir && mariadb-server` and use additional commands to setup the root passord and an admin account with `docker exec  build_hpedevcloud_db_1 /usr/bin/mysqladmin -u root password 'linux1'`. This is left as an exercise for the reader.

But moreover, we now have the start of a micro-service architecture. Each service is isolated in its own container and can be scaled separately. let's try with the web server:

In [ ]:
docker-compose down
docker-compose up --scale hpedevcloud_web=2 -d
docker-compose ps

For that to work, we would need a clustered version of MariaDB on the back-end side, and avoid the port conflict as you can see upper on the front-end side. One way is to manage a load-balancer to receive the requests on that port which we would then redirect to multiple frontend web servers. Let's try !

For that we'll have to:

   1. Add a new container running a haproxy server, with a persistent storage for its configuration and using a private network to communicate with the frontend.
   3. Look at the result
   
More details on how haproxy is working are available at https://en.wikipedia.org/wiki/HAProxy and information on the Docker Hub image at https://hub.docker.com/_/haproxy

In [ ]:
export STUDENTHOME=`dirname $(pwd)`
mkdir -p ../ha

cat > ../ha/haproxy.cfg << EOF
global
    log /dev/log local0
    log localhost local1 notice
    maxconn 2000
    daemon
 
defaults
    log global
    mode http
    option httplog
    option dontlognull
    retries 3
    timeout connect 5000
    timeout client 50000
    timeout server 50000
 
frontend http-in
    bind *:8000
    default_backend webservers
 
backend webservers
    stats enable
    stats auth admin:admin
    stats uri /haproxy?stats
    balance roundrobin
    option httpchk
    option forwardfor
    option http-server-close
    server-template hpedevcloud_web_ 6 hpedevcloud_web:80 check
EOF

cat > docker-compose.yml << EOF
version: '3.5'
services:
  hpedevcloud_web:
    build: .
    image: img_hpedevcloud_web
    volumes:
      - $STUDENTHOME/data:/var/www/html/nextcloud/data
      - $STUDENTHOME/config:/var/www/html/nextcloud/config
    networks:
      nclan:
      proxylan:
        aliases:
          - hpedevcloud_web
    depends_on:
      - hpedevcloud_db
      
  hpedevcloud_db:
    image: mariadb
    command: --transaction-isolation=READ-COMMITTED --binlog-format=ROW --innodb-file-per-table=1 --skip-innodb-read-only-compressed
    restart: always
    environment:
      - MYSQL_ROOT_PASSWORD=password
      - MYSQL_DATABASE=nextcloud
      - MYSQL_USER=nextcloud
      - MYSQL_PASSWORD=nextcloudpwd
    networks:
      - nclan
    volumes:
      - $STUDENTHOME/db:/var/lib/mysql
      
  hpedevcloud_proxy:
    image: haproxy:2.5
    ports:
      - "$HTTPPORT:8000"
    networks:
      - proxylan
    volumes:
      - $STUDENTHOME/ha:/usr/local/etc/haproxy
    depends_on:
      - hpedevcloud_web
      
networks:
  nclan:
    driver: bridge
  proxylan:
    driver: bridge
EOF

docker-compose down
docker-compose up --scale hpedevcloud_web=2
docker-compose ps

It seems to work. All containers are launched and ports seem to work ok. If you log on nextcloud however you may start seeing timeouts, and deconnections or not even being able to login as nextcloud is not cluster aware. Well this is in fact normal.

<br><br>

## <i class="fas fa-2x fa-map-marker-alt" style="color:#551199;"></i>&nbsp;&nbsp;Next Steps

# Conclusion

<h2>Next LAB&nbsp;&nbsp;&nbsp;&nbsp;<a href="4-WKSHP-Conclusion.ipynbb" target="New" title="Next : Conclusion"><i class="fas fa-chevron-circle-right" style="color:#551199;"></i></a></h2>

</br>
 <a href="2-WKSHP-Using-Docker.ipynb" target="New" title="Back: Using Docker"><button type="submit"  class="btn btn-lg btn-block" style="background-color:#551199;color:#fff;position:relative;width:10%; height: 30px;float: left;"><b>Back</b></button></a>
 <a href="3-WKSHP-Using-Docker-Compose.ipynb" target="New" title="Next:Conclusion"><button type="submit"  class="btn btn-lg btn-block" style="background-color:#551199;color:#fff;position:relative;width:10%; height: 30px;float: right;"><b>Next</b></button></a>
